In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark-nlp") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.5") \
    .getOrCreate()
sc = spark.sparkContext

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('words')

[nltk_data] Downloading package stopwords to /home/hadoop/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/hadoop/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [3]:
# get the list of stopwords from nltk
from nltk.corpus import stopwords

eng_stopwords = stopwords.words('english')
eng_stopwords.append('rt')
eng_stopwords.append('qt')
eng_stopwords.append('&amp')
eng_stopwords.append('amp')
eng_stopwords.append('+')
eng_stopwords.append('w')
eng_stopwords.append('today')
eng_stopwords.append('make')
eng_stopwords.append('hear')
eng_stopwords.append('meet')
eng_stopwords.append('see')
eng_stopwords.append('time')
eng_stopwords.append('day')
eng_stopwords.append('watch')
eng_stopwords.append('get')
eng_stopwords.append('im')

In [4]:
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer, 
                                LemmatizerModel, StopWordsCleaner)
from pyspark.ml.feature import CountVectorizer , IDF
from pyspark.ml.clustering import LDA, LDAModel
from pyspark.ml import Pipeline
import pyspark.sql.functions as f
import boto3, os

In [5]:
documentAssembler = DocumentAssembler() \
    .setInputCol('text_no_links') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

# note normalizer defaults to changing all words to lowercase.
# Use .setLowercase(False) to maintain input case.
normalizer = Normalizer() \
    .setInputCols(['token']) \
    .setOutputCol('normalized') \
    .setLowercase(True)

# note that lemmatizer needs a dictionary. So I used the pre-trained
# model (note that it defaults to english)
lemmatizer = LemmatizerModel.pretrained() \
    .setInputCols(['normalized']) \
    .setOutputCol('lemma') \

stopwords_cleaner = StopWordsCleaner() \
    .setInputCols(['lemma']) \
    .setOutputCol('clean_lemma') \
    .setCaseSensitive(False) \
    .setStopWords(eng_stopwords)

# finisher converts tokens to human-readable output
finisher = Finisher() \
    .setInputCols(['clean_lemma']) \
    .setCleanAnnotations(False)

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [6]:
pipeline = Pipeline() \
    .setStages([
        documentAssembler,
        tokenizer,
        normalizer,
        lemmatizer,
        stopwords_cleaner,
        finisher
    ])

In [7]:
# Create sparse vector of counts for LDA input featuresCol
cv = CountVectorizer(inputCol="finished_clean_lemma",
                     outputCol="features", vocabSize=4500,
                     minDF=10)
# IDF to prepare for LDA
idf = IDF(inputCol="features", outputCol="idf")
# LDA model
lda = LDA(k=5, maxIter=40)

In [8]:
# Create Spark-NLP Pipeline
mlPipeline = Pipeline(stages=[
    cv,
    idf,
    lda
])

In [9]:
bangIO = spark.read.csv("s3://502finalprojbucky/InfOpTweets/bangladesh_linked_tweets_csv_hashed.csv",header=True)

In [10]:
#InfOpDF17 = sc.union([]).toDF()

In [12]:
#InfOpDF18 = sc.union([]).toDF()

In [14]:
#InfOpDF19 = sc.union([]).toDF()

In [15]:
InfOpDF = sc.union([InfOpDF17.rdd,InfOpDF18.rdd,InfOpDF19.rdd]).toDF()

In [16]:
def LDA_Prep(DF):
    data = DF.filter(DF["tweet_text"].isNull()==False).filter(DF["tweet_language"]=='en').drop("_c0")
    noLinkText = data.withColumn('text_no_links',f.regexp_replace('tweet_text','http.*($|\s)',''))
    nlpPipeDF = pipeline.fit(noLinkText).transform(noLinkText).select("tweet_text","finished_clean_lemma")
    return nlpPipeDF

In [41]:
def LDA_Eval(mlPiped,time):
    s3 = boto3.resource('s3')
    ldaModel=mlPiped.stages[2]
    ll = ldaModel.logLikelihood(LDA_DF)
    lp = ldaModel.logPerplexity(LDA_DF)
    with open('results.txt','w') as file:
        file.write("Learned topics for "+str(time)+" congress tweets (as distributions over vocab of "
                    + str(ldaModel.vocabSize())+ " words):\n\n")
        file.write("The lower bound on the log likelihood of the entire corpus: " + str(ll))
        file.write("\nThe upper bound on perplexity: " + str(lp))
        file.write("\n")
    #ldaModel.describeTopics(5).show(truncate=False)
    # Output topics. Each is a distribution over words (matching word count vectors)
    topics = ldaModel.describeTopics(15)
    topics_rdd = topics.rdd
    vocab = mlPiped.stages[0].vocabulary
    topics_words = topics_rdd\
           .map(lambda row: row['termIndices'])\
           .map(lambda idx_list: [vocab[idx] for idx in idx_list])\
           .collect()
    with open('results.txt','a') as file:
        for idx, topic in enumerate(topics_words):
            file.write("\n----------")
            file.write("\ntopic: "+str(idx))
            file.write("\n----------\n")
            for word in topic:
                file.write(word+" | ")
        file.write("\n========================================================================\n")
    s3.meta.client.upload_file('results.txt', '502finalprojbucky',
                               'InfoOpresults/results'+str(time).replace(" ","")+'.txt')

In [18]:
# Prep the data for LDA
prepDF = LDA_Prep()

In [19]:
# Fit the model
mlPipeFit = mlPipeline.fit(prepDF)

In [20]:
# Transform the data
LDA_DF = mlPipeFit.transform(prepDF)

In [42]:
# Write summary of the results to a text file in s3 bucket
LDA_Eval(mlPipeFit,"")

In [ ]:
#spark.stop()